In [8]:
import cobra
import time
import numpy as np
import more_itertools as mi
import multiprocessing as mp
import random

import gc

In [9]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [10]:
from cobra import Model, Reaction, Metabolite

In [11]:
start=time.time()

In [12]:
cobra.Configuration().solver='cplex'

In [13]:
%%time
model=cobra.io.load_matlab_model("./models/Human_1_16.mat")

CPU times: user 13.3 s, sys: 146 ms, total: 13.5 s
Wall time: 13.5 s


In [14]:
model2=model.copy()

In [15]:
model2

Name,ihuman
Memory address,7f56ed5e4978
Number of metabolites,8499
Number of reactions,13085
Number of genes,2897
Number of groups,0
Objective expression,1.0*MAR13082 - 1.0*MAR13082_reverse_11d67
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [16]:
for j in range(len(model2.reactions)):
    r=model2.reactions[j]
    if r.id=="MAR13082":
        biomass=j
biomass

12906

In [17]:
model2.objective=model2.reactions[biomass]
model2.pobjective_direction="max"
maxBio=model2.slim_optimize()
maxBio

138.46657955800086

In [18]:
def identificaTipo(nombre):
    if "(" in nombre:
        indice1=nombre.index("(")
        nombre2=nombre[indice1+1:]
        indice2=nombre2.index(")")
        nombre3=nombre2[:indice2] 
        if " and " in nombre3:
            tipo="SOP"
        if " or " in nombre3:
            tipo="POS"
    else:
        tipo="SOP"
    return(tipo)

def toPOS(nombre,tope=3):

    productos=set()    
    tipo=identificaTipo(nombre)

    if tipo=="SOP":
        sumandos=nombre.split(" or ")
        for sumando in sumandos:
            if " and " in sumando:
                casos=sumando.split(" and ")
                miCaso=set()
                for caso in casos:
                    caso2=caso.replace("(","")
                    caso3=caso2.replace(")","")
                    caso4=caso3.replace(" ","")
                    miCaso.add(caso4)

                productos.add(frozenset(miCaso))
            else:
                productos.add(frozenset([sumando]))

        CS=set([frozenset()])
        for producto in productos:
            CS2=set()
            for C in CS:
                if len(set(C).intersection(producto))==0:
                    for a in producto:
                        C2=set(C.copy())
                        C2.add(a)
                        if len(C2)<=tope:
                            CS2.add(frozenset(C2))
                else:
                    CS2.add(C)
            CS=CS2

        CS3=set()
        for C in CS:
            bien=True
            for C2 in CS:
                if not C==C2 and C2.issubset(C):
                    bien=False
                    break
            if bien:
                CS3.add(C)
    else:
        CS2=set()
        productos = nombre.split(" and ")
        for producto in productos:
            CS=set()
            componentes=producto.split(" or ")
            for C in componentes:
                C2=C.replace("(","")
                C3=C2.replace(")","")
                C4=C3.replace(" ","")
                CS.add(C4)
            CS2.add(frozenset(CS))
        CS3=set()
        for C in CS2:
            bien=True
            for C2 in CS2:
                if not C==C2 and C2.issubset(C):
                    bien=False
                    break
            if bien:
                CS3.add(C)
    return(CS3)

def asociar(modelo,tope=4):
    verbose=False
    total=0
    asociados=dict()
    reacciones=dict()

    for num in range(len(modelo.reactions)):
        nombre=modelo.reactions[num].gene_reaction_rule
        CS3=toPOS(nombre,tope)
        k=0
        while k<=tope:
            mas=0
            nivel=set([C for C in CS3 if len(C)==k and not C==frozenset({''})])
            if len(nivel)>0:
                mios=set()
                for caso in nivel:
                    mio=set()
                    for a in caso:
                        mio.add(a)
                    mios.add(frozenset(mio))

                asociados[(modelo.reactions[num],k)]=mios

                for caso in mios:
                    if not caso in reacciones:
                        reacciones[caso]=set([modelo.reactions[num]])
                    else:
                        reacciones[caso].add(modelo.reactions[num])
                mas+=len(nivel)
            total+=mas
            k+=1    
        if verbose:
            print(num,len(modelo.reactions[num].genes),mas)

    return(asociados,reacciones)
    
    
def soporteG(soporte,tope,modelo=model2):
    sopG=set()
    for j in soporte:
        r=modelo.reactions[j]
        for k in range(1,tope+1):
            if (r,k) in asociados:
                sopG.update(asociados[(r,k)])

    sopG1=set([frozenset(C) for C in sopG])
        
    sopG2=set()
    for caso in sopG1:
        poner=True
        for caso2 in sopG1:
            if not caso==caso2 and caso2.issubset(caso) and len(caso2)>0:
                poner=False
                break
        if poner:
            sopG2.add(caso)
    return(sopG2)

def chequea(inicio,fin):
    respuestas=set()
    for i in range(inicio,fin):
        if i<len(finales):
            C=finales[i]
            vale=True
            if not C.isdisjoint(CSs1):
                vale = False
            else:
                if C in powers:
                    po=powers[C]
                else:
                    po=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                    powers[C]=po

                if not po.isdisjoint(todosMCS):
                    vale=False

                if vale:
                    for sop in listaSoportes[indice[C]:]:
                        if po.isdisjoint(sopG[sop]):           
                            vale=False
                            break
            if vale:
                respuestas.add(C)

    return(respuestas)


In [19]:
def reduceLista2():
    LOld=lista2.copy()
    L1=set([C for C in lista2 if len(C)==1])
    lista2.difference_update(L1)
    L2=set([C for C in lista2 if len(C)==2])
    lista2.difference_update(L2)
    L3=set([C for C in lista2])

    for C in L1:
        mios1=set([C2 for C2 in L2 if not C.issubset(C2)])
        mios2=set([C3 for C3 in L3 if not C.issubset(C3)])
        L2.intersection_update(mios1)
        L3.intersection_update(mios2)

    Gs=set([g for C in L3 for g in C])    
    for C in L2:
        mios2=set([frozenset(set(C).union(set([g]))) for g in Gs])
        L3.difference_update(mios2)    
    L1.update(L2)
    L1.update(L3)
    LOld.difference_update(L1)
    return L1




In [20]:
def CSV2(modelo2,tope,misReacciones,misValores,verbose,evita):
    global tiempos
    global todosMCS
    global evitar
    global misSoportes
    global listaMias
    global listaFinales
    global sopG, powers
    global modelo
    
    global miLista2
    global listaL2
    
    global lista3
    
    global finales
    global CSs1
    global nuevos
    
    global listaSoportes, indice, iMio, lista2
    
    modelo=modelo2
    
    listaMias=[]
    
    
    evitar=evita
    listaMCS=[]
    listaMCS.append(set())
    
    listaCandidatos=[]
    listaCandidatos.append(set([frozenset()]))
    todosMCS=set()
    cutsets=[set()]
    candidatos=[set()]

    ts=0
    
    for i in range(1,tope+1):
        if i==1:      
            
            
            st=time.time()
            cpus=mp.cpu_count()
            cpus=64
            miL2=set()
            listaGenes=list(modelo.genes)

            k=0
            argus=[]
            num=int(np.floor(len(modelo.genes)/cpus))
            if num==0:
                num=1
            while k*num<len(listaGenes):
                argus.append((k*num,(k+1)*num))
                k+=1

            with mp.Pool(processes=cpus) as mp_pool:
                results=mp_pool.starmap(analizaGenes,argus)

            CS0=set()
            
            for result in results:
                CS0.update(result[0])
                misSoportes.update(result[1])
                
            print(len(CS0),len(misSoportes))
            todosMCS.update(CS0)    
            cutsets.append(CS0)
            
            for sop in misSoportes:
                sopG[sop]=soporteG(sop,tope,modelo)
            
            print("Final Fase 1 con",len(cutsets[1]),"cutsets")

            CSs1=set([g for caso in CS0 for g in caso])
            
            
            for sop in sopG:
                sopG[sop]=reduce(sopG[sop])
            print(time.time()-st)
            print()
       
        ############################## Final Fase 1 ############################################
    
        else:
            print("Inicio de Fase",i,"con",len(misSoportes),"soportes")
            iMio=i
            
            ############################## Parte 1.- Berge #################################### 
            st1=time.time()
            CST2=set()
            lista=set([frozenset()])
            descartados=set()
            cont2=0
            long=dict()
            indice=dict()
            analizados=set()
            
            for sop in misSoportes:
                long[sop]=len(sopG[sop])
            sortedSupports= sorted(long.items(), key=lambda x:x[1])
            finales=set()
            listaSoportes=[sortedSupports[t][0] for t in range(len(misSoportes))]
            
            for t in range(len(misSoportes)):
                ################ Para cada soporte #################
                item=sortedSupports[t]
                sop=item[0]
                
                
                
                lista2=set()
                for caso in lista:
                    sst=time.time()
                    if caso in powers:
                        po=powers[caso]
                    else:
                        po=set([frozenset(a) for a in mi.powerset(caso) if len(a)>0])
                        powers[caso]=po
                    esta=(len(po.intersection(sopG[sop]))>0)
                    ts+=(time.time()-sst)
                    if esta:
                        lista2.add(caso)
                    else:
                        for caso2 in sopG[sop]:
                            if len(caso.union(caso2))<iMio:
                                nuevo=caso.union(caso2)
                                lista2.add(nuevo)
                            if len(caso.union(caso2))==iMio:
                                nuevo=caso.union(caso2)
                                finales.add(nuevo)
                                indice[nuevo]=t

                lista2=reduceLista2()
                lista4=set()
                for C in lista2:
                    sst=time.time()
                    if C in powers:
                        po2=powers[C]
                    else:
                        po2=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                        powers[C]=po2
                    if po2.isdisjoint(todosMCS) and evitar.isdisjoint(C):
                        lista4.add(C)
                    ts+=(time.time()-sst)
                cont2+=1
                lista=lista4
                lista3=[C for C in lista if len(C)<i]
                if verbose:
                    print("Mal",len(lista3))

            
            print("Finales",len(finales))
                    #break
            ########################## He terminado de procesar los soportes ###############
            
            if i==tope:
                print("Fase1",time.time()-st1)
            st2=time.time()
            
            if verbose:
                print("Parte paralela")
            
            lista3=[C for C in lista if len(C)<i]
            
            
            
            misFinales=list(finales)
            if verbose:
                print("Total",len(misFinales))
            lista=set()
            numLista=8*10**6
            r=0
            while r*numLista<len(misFinales):
                maximo=(r+1)*numLista
                if len(misFinales)<maximo:
                    maximo=len(misFinales)
                finales=misFinales[r*numLista:maximo]
                if verbose:
                    print(r,"Finales",r*numLista,maximo)
                r+=1
                

                finales=list(finales)


                cpus=mp.cpu_count()
                cpus=64
                k=0
                argus=[]
                num=int(np.floor(len(finales)/cpus))
                if num==0:
                    num=1
                while k*num<len(finales):
                    argus.append((k*num,(k+1)*num))
                    k+=1

                with mp.Pool(processes=cpus) as mp_pool:
                    results=mp_pool.starmap(chequea,argus)

                for result in results:
                    lista.update(result)
                

            del misFinales
            del results
            del finales
            del listaSoportes
            gc.collect()

            if i==tope:
                print("Fase2",time.time()-st2)
            st3=time.time()
            
            cont2=1
            
            listaL2=list(lista)
            
            
            nuevos=set()
            lista2=set()
            cpus=mp.cpu_count()
            cpus=64

            k=0
            argus=[]
            num=int(np.floor(len(listaL2)/cpus))
            if num==0:
                num=1
            while k*num<len(listaL2):
                argus.append((k*num,(k+1)*num))
                k+=1

            with mp.Pool(processes=cpus) as mp_pool:
                results=mp_pool.starmap(analizaSalto,argus)

            for result in results:
                lista2.update(result[0])
                misSoportes.update(result[1])
                nuevos.update(result[1])
                for sop in result[1]:
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
                cont2+=1



            #Refiltrar
            listaL2=list(lista2)
            for C in lista2:
                if not C in powers:
                    powers[C]=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])

            lista3=set()
            k=0
            argus=[]
            num=int(np.floor(len(listaL2)/cpus))
            if num==0:
                num=1
            while k*num<len(listaL2):
                argus.append((k*num,(k+1)*num))
                k+=1

            with mp.Pool(processes=cpus) as mp_pool:
                results=mp_pool.starmap(filtra,argus)

            for result in results:
                lista3.update(result)

            lista2=lista3
            listaL2=list(lista2)


            if verbose:
                print()
                print("Lista2",len(lista2))
                print()
            
            cutsetsT=set()
            
            if i==tope:
                print("Fase3",time.time()-st3)
                miLista2=lista2
            st4=time.time()
            
            if verbose:
                print("Sin salto")
            descartados=set()
            cont=0
            if len(lista2)<100:
                cpus=2
            elif len(lista2)<1000:
                cpus=10
            else:
                cpus=64
            miLista2=list(lista2)
            listaT=set()

            k=0
            argus=[]
            num=int(np.floor(len(miLista2)/cpus))
            if num==0:
                num=1
            while k*num<len(miLista2):
                argus.append((k*num,(k+1)*num))
                k+=1

            with mp.Pool(processes=cpus) as mp_pool:
                results=mp_pool.starmap(analiza,argus)

            for result in results:
                listaT.update(result[0])
                misSoportes.update(result[1])
                for sop in result[1]:
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
            
            cutsets.append(set(listaT))
            if i==tope:
                print("Fase4",time.time()-st4)
            todosMCS.update(listaT)   
        if i>1:
            tiempos.append([st1,st2,st3])
    sopG=dict()
    powers=dict()
    misSoportes=set()
    return(cutsets)
                        

In [21]:
def reduce(sG):
    sopG1=set([frozenset(C) for C in sG])
    sopG2=set()
    for caso in sopG1:
        poner=True
        for caso2 in sopG1:
            if not caso==caso2 and caso2.issubset(caso) and len(caso2)>0:
                poner=False
                break
        if poner:
            sopG2.add(caso)
    return(sopG2)

In [22]:
def analizaSalto(inicio,fin):
    misSoportes2=set()
    
    descartados=set()
    miLista2=set()
    
    i2=inicio
    while i2 < fin:
        casos=[]
        contCasos=0
        for k in range(i2,fin):
            if k<len(listaL2) and not listaL2[k] in descartados:
                casos.append(listaL2[k])
                contCasos+=1
            if contCasos>=salto:
                break
        i2=k+1

        cont3=0
        if len(casos) > 0:
            with modelo:
                for caso in casos:
                    descartados.add(caso)
                    for g in caso:
                        modelo.genes.get_by_id(g).knock_out()
                for i in range(len(misReacciones)):
                    modelo.reactions[misReacciones[i]].bounds=misValores[i]
                
                sol=modelo.slim_optimize(error_value=-1)
                if sol==-1:
                    for caso in casos:
                        miLista2.add(caso)
                else:
                    sol2=modelo.optimize()
                    sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8])
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
                    misSoportes2.add(sop)
                    for C in set(listaL2).difference(descartados): 
                        sst=time.time()
                        if C in powers:
                            sub=powers[C]
                        else:
                            sub=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                            powers[C]=sub
                        if sub.isdisjoint(sopG[sop]):
                            descartados.add(C)
                        

    return([miLista2,misSoportes2])

In [23]:
def analiza(inicio,fin):    
    misSoportes2=set()
    descartados=set()
    CSs=set()
    for caso in miLista2[inicio:fin]:
        if not caso in descartados:
            descartados.add(caso)
            with modelo:
                for g in caso:
                    modelo.genes.get_by_id(g).knock_out()
                for i in range(len(misReacciones)):
                    modelo.reactions[misReacciones[i]].bounds=misValores[i]
                sol=modelo.slim_optimize(error_value=-1)
                if sol==-1:
                    CSs.add(caso)
                elif iMio<tope:
                    sol2=modelo.optimize()
                    sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8])
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
                    misSoportes2.add(sop)
                    for C in set(miLista2).difference(descartados): 
                        sst=time.time()
                        sub=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                        if sub.isdisjoint(sopG[sop]):
                            descartados.add(C)
                        
    return([CSs,misSoportes2])

In [24]:
def analizaGenes(inicio,fin):
    correctas=set()
    soportes=set()
    for i in range(inicio,fin):
        if i<len(listaGenes):
            g=modelo.genes.get_by_id(listaGenes[i].id)
            with modelo:
                g.knock_out()
                for i in range(len(misReacciones)):
                    modelo.reactions[misReacciones[i]].bounds=misValores[i]
                sol=modelo.slim_optimize(error_value=-1)
                if sol==-1:
                    correctas.add(frozenset([g.id]))
                else:
                    sol2=modelo.optimize(objective_sense="maximize")
                    sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8])
                    soportes.add(sop)
    return([correctas,soportes])

In [25]:
def filtra(inicio,fin):    
    CSs=set()
    for caso in listaL2[inicio:fin]:
        vale=True
        for sop in nuevos:
            po=powers[caso]
            sG=sopG[sop]
            if po.isdisjoint(sG):
                vale=False
                break
        if vale:
            CSs.add(caso)
                        
    return(CSs)

In [26]:
misReacciones, misValores = [biomass],[[0.01,1000]]

In [27]:
times=[]
tiempos=[]
for intento in range(1):
    start=time.time()
    modelo=model2.copy()
    listaGenes=list(modelo.genes)
    misSoportes=set()
    tope=4
    salto=14
    asociados,reacciones=asociar(modelo,tope)
    sopG=dict()
    powers=dict()
    CS1=CSV2(modelo,tope,misReacciones,misValores,False,set())
    t=time.time()-start
    times.append(t)
    print("Tiempo",t)

92 279
Final Fase 1 con 92 cutsets
19.85220217704773

Inicio de Fase 2 con 279 soportes
Finales 20020
Inicio de Fase 3 con 437 soportes
Finales 874267
Inicio de Fase 4 con 535 soportes
Finales 45322730
Fase1 166.38722586631775
Fase2 165.23437547683716
Fase3 75.43327760696411
Fase4 2.5323452949523926
Tiempo 459.19034075737


In [28]:
total=0
for i,C in enumerate(CS1):
    print(i,len(C))
    total+=len(C)
print(total)

0 0
1 92
2 14
3 15
4 12
133
